<a href="https://colab.research.google.com/github/readthroughmyglasses/FYP_QUORA/blob/master/Hierarchical_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')
# drive.mount("/content/drive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


**Setting the path for data**

In [0]:
path = '/content/drive/My Drive/FINAL DESTINATION/DATA/train.csv'

In [0]:
# data_folder = '/content/drive/My Drive/FINAL DESTINATION/DATA/'
data_folder = '/content/drive/My Drive/DS/DATA/'
requirements = '/content/drive/My Drive/FINAL DESTINATION/DATA/requirements.txt'

In [0]:
input_folder = '/content/drive/My Drive/DS'

In [0]:
#!python -m spacy download en_vectors_web_lg
!pip install utils

!pip install jellyfish
!pip install testfixtures
!pip install vocabulary

!pip freeze > '/content/drive/My Drive/FINAL DESTINATION/DATA/requirements.txt'

!pip install hyperas

/bin/bash: /content/drive/My Drive/FINAL DESTINATION/DATA/requirements.txt: No such file or directory


In [0]:
import io
import seaborn as sns
import spacy

%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from math import*

import sys
import os 
import re
import string

import jellyfish
import gensim
from gensim.models import *
from gensim.parsing.preprocessing import preprocess_string, strip_tags, strip_multiple_whitespaces, remove_stopwords, stem_text
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec

import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
nltk.download('universal_tagset')
nltk.download('punkt')

import utils
import vocabulary
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report, log_loss
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import GridSearchCV
from importlib import reload
from textblob import TextBlob
reload(utils)

import time
import timeit
import itertools

import keras
from keras import layers, Model, models, initializers
from keras.engine.topology import Layer
from keras import backend as K
from keras.optimizers import Adam,RMSprop, Nadam, SGD
from keras.callbacks import EarlyStopping

import sys
sys.path.append('/content/drive/My Drive/FINAL DESTINATION/DATA/src/utils/')
from plotting import plot_roc
# import src.utils.dl_utils
# from src.utils.dl_utils import *

from dl_utils import init_session, data, plot_optimization_history

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform, loguniform, lognormal, normal
import gc

import pickle

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Using TensorFlow backend.


ModuleNotFoundError: ignored

In [0]:
dtypes = {'id': 'int64','qid1': 'int64','qid2': 'int64','question1': 'object','question2': 'object','is_duplicate': 'int64'}
df = pd.read_csv('/content/drive/My Drive/FINAL DESTINATION/DATA/train.csv', sep=',', header=0, usecols=dtypes.keys(), skipinitialspace=True, skip_blank_lines=True, encoding='utf-8')
df = df.set_index('id')
df = df.dropna()

In [0]:
X = df.drop(columns=['is_duplicate'])
y = df['is_duplicate']
X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.5, random_state=42)
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 202143 entries, 79039 to 121959
Data columns (total 4 columns):
qid1         202143 non-null int64
qid2         202143 non-null int64
question1    202143 non-null object
question2    202143 non-null object
dtypes: int64(2), object(2)
memory usage: 7.7+ MB


In [0]:
import pickle
pickle.dump(X_train, open(input_folder+'X_train.p', 'wb'))
pickle.dump(y_train, open(input_folder+'y_train.p', 'wb'))
pickle.dump(X_test, open(input_folder+'X_test.p', 'wb'))
pickle.dump(y_test, open(input_folder+'y_test.p', 'wb'))

In [0]:
del X,y,df,X_train,X_test, y_train, y_test

In [0]:
X_train = pickle.load(open(input_folder+'X_train.p', 'rb'))
X_test = pickle.load(open(input_folder+'X_test.p', 'rb'))
y_train = pickle.load(open(input_folder+'y_train.p', 'rb'))
y_test = pickle.load(open(input_folder+'y_test.p', 'rb'))

In [0]:
!python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [0]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 826.9MB 1.1MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.1.0-cp36-none-any.whl size=828255078 sha256=73c4a58433a69395e6f25605ffa125a7fdd838b9aa671f9ecc1d0c018d1d90c3
  Stored in directory: /tmp/pip-ephem-wheel-cache-64bj8kvd/wheels/b4/d7/70/426d313a459f82ed5e06cc36a50e2bb2f0ec5cb31d8e0bdf09
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [0]:
import en_core_web_lg
nlp = en_core_web_lg.load()

In [0]:
def create_dataset(nlp, X, num_oov, max_length, norm_vectors = True):
    len_q1 = X['question1'].size
    sents = pd.concat([X['question1'], X['question2']]).values
    
    # the extra +1 is for a zero vector represting NULL for padding
    num_vectors = max(lex.rank for lex in nlp.vocab) + 2 
    
    # create random vectors for OOV tokens
    oov = np.random.normal(size=(num_oov, nlp.vocab.vectors_length))
    oov = oov / oov.sum(axis=1, keepdims=True)
    
    vectors = np.zeros((num_vectors + num_oov, nlp.vocab.vectors_length), dtype='float32')
    vectors[num_vectors:, ] = oov
    for lex in nlp.vocab:
        if lex.has_vector and lex.vector_norm > 0:
            vectors[lex.rank + 1] = lex.vector / lex.vector_norm if norm_vectors == True else lex.vector
            
    sents_as_ids = []
    for sent in sents:
        doc = nlp(sent)
        word_ids = []
        
        for i, token in enumerate(doc):
            # skip odd spaces from tokenizer
            if token.has_vector and token.vector_norm == 0:
                continue
                
            if i > max_length:
                break
                
            if token.has_vector:
                word_ids.append(token.rank + 1)
            else:
                # if we don't have a vector, pick an OOV entry
                word_ids.append(token.rank % num_oov + num_vectors) 
                
        # there must be a simpler way of generating padded arrays from lists...
        word_id_vec = np.zeros((max_length), dtype='int')
        clipped_len = min(max_length, len(word_ids))
        word_id_vec[:clipped_len] = word_ids[:clipped_len]
        sents_as_ids.append(word_id_vec)        
        
    return vectors, np.array(sents_as_ids[:len_q1]), np.array(sents_as_ids[len_q1:])

In [0]:

def create_embedding(vectors, max_length, projected_dim):
    return models.Sequential([
        layers.Embedding(
            vectors.shape[0],
            vectors.shape[1],
            input_length=max_length,
            weights=[vectors],
            trainable=False),
        
        layers.TimeDistributed(
            layers.Dense(projected_dim,
                         activation=None,
                         use_bias=False))
    ])

In [0]:

w2v, q1_train_w2v, q2_train_w2v = create_dataset(nlp, X_train, 100, 50, True)

pickle.dump(w2v, open(data_folder+'w2v.p', 'wb'))
pickle.dump(q1_train_w2v, open(data_folder+'q1_train_w2v.p', 'wb'))
pickle.dump(q2_train_w2v, open(data_folder+'q2_train_w2v.p', 'wb'))

In [0]:
_, q1_test_w2v, q2_test_w2v = create_dataset(nlp, X_test, 100, 50, True)


pickle.dump(q1_test_w2v, open(data_folder+'q1_test_w2v.p', 'wb'))
pickle.dump(q2_test_w2v, open(data_folder+'q2_test_w2v.p', 'wb'))

In [0]:
w2v = pickle.load(open(data_folder+'w2v.p', 'rb'))
q1_train_w2v = pickle.load(open(data_folder+'q1_train_w2v.p', 'rb'))
q2_train_w2v = pickle.load(open(data_folder+'q2_train_w2v.p', 'rb'))
q1_test_w2v = pickle.load(open(data_folder+'q1_test_w2v.p', 'rb'))
q2_test_w2v = pickle.load(open(data_folder+'q2_test_w2v.p', 'rb'))

In [0]:
from keras.callbacks import ModelCheckpoint

In [0]:
from keras.layers import Layer, RNN

class Attention_Layer(Layer):

    def __init__(self, output_dim):
        self.output_dim = output_dim
        super(Attention_Layer, self).__init__()

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.W = self.add_weight(name='W', 
                                      shape=(input_shape[-1], self.output_dim),
                                      initializer='uniform',
                                      trainable=True)
        self.b = self.add_weight(name='b', 
                                      shape=(self.output_dim,),
                                      initializer='uniform',
                                      trainable=True)
        self.u = self.add_weight(name='u', 
                                      shape=(self.output_dim,1),
                                      initializer='uniform',
                                      trainable=True)
        super(Attention_Layer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, h_it):        
        u_it = K.tanh(K.bias_add(K.dot(h_it, self.W), self.b))
        att_weights = K.dot(u_it, self.u)
        exp_weights = K.exp(att_weights)
        sum_weights = K.sum(exp_weights, axis=1, keepdims=True)
        alpha_it = exp_weights/sum_weights
        return K.sum(h_it*alpha_it, axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])


In [0]:
from keras.callbacks import LambdaCallback
def build_hier_model(vectors, optimizer, learn_rate, dropout_rate1, dropout_rate2, max_length=50, num_hidden=200, num_classes=1, 
                projected_dim=200):
    # optimizer=params['optimizer']
    # learn_rate=params['learn_rate']
    # dropout_rate1=params['dropout_rate1']
    # dropout_rate2=params['dropout_rate1']
    #K.clear_session()
    # input    
    model_input = layers.Input(shape=(2, max_length), dtype='int32')
    
    # embeddings (projected)
    embed = create_embedding(vectors, max_length, projected_dim)
    
    # step 1: word encoder
    word_sequence_input = layers.Input(shape=(max_length,), dtype='int32')
    h_w = layers.Bidirectional(layers.GRU(num_hidden, dropout=dropout_rate1, return_sequences=True))(embed(word_sequence_input))
    
    # step 2: word attention
    s_w = Attention_Layer(num_hidden)(h_w)
    word_encode_attend = Model(word_sequence_input, s_w)
    
    # step 3: sentence encoder
    sent_encode_attend = layers.TimeDistributed(word_encode_attend)(model_input)
    h = layers.Bidirectional(layers.GRU(num_hidden, dropout=dropout_rate2, return_sequences=True))(sent_encode_attend)
    
    # step 4: sentence attention
    v = Attention_Layer(num_hidden)(h)

    out = layers.Dense(num_classes, activation='sigmoid', use_bias=True)(v)
    
    model = Model(model_input, out)
    
    if optimizer == 'sgd':
        opt = SGD(lr=learn_rate)
    elif optimizer == 'adam':
        opt = Adam(lr=learn_rate)
    elif optimizer == 'rmsprop':
        opt = RMSprop(lr=learn_rate)
    else:
        opt = Nadam(lr=learn_rate)
    
    model.compile(optimizer=opt,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [0]:
from hyperopt import fmin, tpe, hp, Trials
from hyperopt import STATUS_OK

x_train, y_train, x_test, y_test = data()
w2v = pickle.load(open(data_folder+'w2v.p', 'rb'))

In [0]:
from keras.callbacks import LearningRateScheduler
# function for exponential decay (used for laerning rate)
def model2_exp_decay(init_lr, k):
    def _exp_decay(epoch):
        lrate = init_lr * np.exp(-k*epoch)
        return lrate
    return _exp_decay

In [0]:
checkpoint_path='/content/drive/My Drive/DS/DATA/model2-{epoch:02d}-{val_acc:.2f}.hdf5'

x_train
y_train

337571    0
79039     0
12796     1
26184     0
214725    0
         ..
259178    0
365838    1
131932    1
146867    0
121958    1
Name: is_duplicate, Length: 202145, dtype: int64

In [0]:
# optimization function which will be called by hyperopt for each combination of variable parameters
def opt_deep_fn(params):
    K.clear_session()
    # this is needed to free up gpu memory after every evaluation
    gc.collect()
    model = build_hier_model(vectors=w2v, optimizer='adam', 
                             learn_rate= 0.01, dropout_rate1=params['dropout_rate1'], 
                             dropout_rate2=params['dropout_rate2'], max_length=50, projected_dim=200, 
                             num_classes=1, num_hidden=200)
    print_weights = LambdaCallback(on_epoch_end = lambda batch,logs:print(model.layers[0].get_weights()))
    early_stopping = EarlyStopping(monitor='val_loss', patience=10)
    learn_rate = LearningRateScheduler(model2_exp_decay(params['learn_rate'], params['decay']))
    model_checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, mode='min')

    result = model.fit(x_train, y_train.values, batch_size=128, epochs=20,
      validation_split=0.33, callbacks=[early_stopping, learn_rate,model_checkpoint,print_weights],verbose=2)
    validation_acc = np.amax(result.history['val_acc']) 
    print('Best validation accuracy of eval run:', validation_acc)
    #print('history of eval run:', result.history)
    return {'loss': -validation_acc, 'status': STATUS_OK,
            'train_acc': np.amax(result.history['acc'])}

In [0]:
# search space for all variable hyperparameters
search_deep_space = {
                'learn_rate':hp.loguniform('learn_rate',-6,-3),
                'dropout_rate1':hp.uniform('dropout_rate1',0, 1),
                'dropout_rate2':hp.uniform('dropout_rate2',0, 1),
                'decay':hp.uniform('decay',0, 1)
                }

In [0]:
init_session()
model2_run2_trials=Trials()

# hyperopt optimization
model2_run2_best = fmin(opt_deep_fn,
                        search_deep_space,
                          algo=tpe.suggest,
                          max_evals=10,
                           trials=model2_run2_trials)

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5

Train on 135437 samples, validate on 66708 samples
Epoch 1/20
 - 196s - loss: 0.6659 - acc: 0.6274 - val_loss: 0.6597 - val_acc: 0.6300

[]
Epoch 2/20
 - 197s - loss: 0.6616 - acc: 0.6315 - val_loss: 0.6612 - val_acc: 0.6300

[]
Epoch 3/20
 - 192s - loss: 0.6641 - acc: 0.6281 - val_loss: 0.6594 - val_acc: 0.6300

[]
Epoch 4/20
 - 197s - loss: 0.6629 - acc: 0.6316 - val_loss: 0.6751 - val_acc: 0.6300

[]
Epoch 5/20
 - 193s - loss: 0.6627 - acc: 0.6313 - val_loss: 0.6888 - val_acc: 0.6300

[]
Epoch 6/20
 - 194s - loss: 0.6623 - acc: 0.6314 - val_loss: 0.6904 - val_acc: 0.6300

[]
Epoch 7/20
 - 192s - loss: 0.6632 - acc: 0.6301 - val_loss: 0.6709 - val_acc: 0.6300

[]
Epoch 8/20
 - 192s - loss: 0.6620 

KeyboardInterrupt: ignored

NameError: ignored

In [0]:
checkpointer = ModelCheckpoint(filepath='keras_weights.hdf5', 
                                   verbose=1, 
                                   save_best_only=True)

In [0]:
print("Best performing model hyper-parameters:")
print(model2_run2_best)

In [0]:
K.clear_session()
m2 = build_hier_model(vectors=w2v, optimizer='rmsprop', 
                             learn_rate=0.0, dropout_rate1=  , 
                             dropout_rate2=  , max_length=50, projected_dim=200, 
                             num_classes=1, num_hidden=200)
#early_stopping = EarlyStopping(monitor='val_loss', patience=4)
m2.summary()

In [0]:
learn_rate = LearningRateScheduler(model2_exp_decay(  ,  ))
result = m2.fit(x_train, y_train, batch_size= 128, epochs=40,
  validation_split=0.1, callbacks=[learn_rate])